In [1]:

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
import pandas as pd
import json

# Environment variables

PASS = os.getenv("PASS")
DATABASE = os.getenv("DATABASE")
USER = os.getenv("USER")
HOST = os.getenv("HOST")
DATABASE_CLIENT = os.getenv("DATABASE_CLIENT")
PORT = os.getenv("PORT")

In [2]:
# Database connection
def create_db_connection(password):
    DATABASE_URL = f"{DATABASE_CLIENT}://{USER}:{password}@{HOST}:{PORT}/{DATABASE}"
    engine = create_engine(DATABASE_URL)
    Session = sessionmaker(bind=engine)
    return engine, Session()


In [3]:
engine, session = create_db_connection(PASS)

In [4]:
from typing import Dict

In [5]:
def parse_nan_values(dataframe: pd.DataFrame) -> pd.DataFrame:
    return dataframe.fillna("")


def parse_numeric_values(dataframe: pd.DataFrame) -> pd.DataFrame:
    for column in dataframe.select_dtypes(include=["number"]).columns:
        dataframe[column] = dataframe[column].round(2)
    return dataframe


In [6]:
def execute_query(query: str) -> Dict:
    try:
        df_result = pd.read_sql_query(query, engine)
        df_result = parse_nan_values(df_result)
        df_result = parse_numeric_values(df_result)
        return df_result.to_dict(orient="records")
    except Exception as e:
        raise e


In [7]:


def dump_to_json(data: Dict) -> str:
    return json.dumps(data, indent=4)

In [35]:
res = execute_query("""SELECT a.airline, COUNT(*) AS total_flights, AVG(distance) AS avg_distance
FROM flights f
JOIN airlines a ON f.airline = a.iata_code
WHERE f.year = 2015
GROUP BY a.airline
ORDER BY a.airline;


""")

In [36]:

data = dump_to_json(res)
data

'[\n    {\n        "airline": "Alaska Airlines Inc.",\n        "total_flights": 167,\n        "avg_distance": 1258.01\n    },\n    {\n        "airline": "American Airlines Inc.",\n        "total_flights": 632,\n        "avg_distance": 1035.42\n    },\n    {\n        "airline": "American Eagle Airlines Inc.",\n        "total_flights": 260,\n        "avg_distance": 433.22\n    },\n    {\n        "airline": "Atlantic Southeast Airlines",\n        "total_flights": 512,\n        "avg_distance": 453.05\n    },\n    {\n        "airline": "Delta Air Lines Inc.",\n        "total_flights": 747,\n        "avg_distance": 852.72\n    },\n    {\n        "airline": "Frontier Airlines Inc.",\n        "total_flights": 66,\n        "avg_distance": 979.48\n    },\n    {\n        "airline": "Hawaiian Airlines Inc.",\n        "total_flights": 64,\n        "avg_distance": 664.92\n    },\n    {\n        "airline": "JetBlue Airways",\n        "total_flights": 235,\n        "avg_distance": 1041.18\n    },\n   

'[\n    {\n        "avg_departure_delay": 10.61\n    }\n]'